# Code for plots to help select flies for downstream 10x processing

# 1. load packages

devtools::install_github("shaliulab/behavr@deployment")
devtools::install_github("shaliulab/scopr@deployment")
devtools::install_github("shaliulab/sleepr@deployment")
devtools::install_github("shaliulab/ggetho@deployment")

library(devtools)
install_github("rethomics/ggetho")

library(behavr)
library(scopr)
library(sleepr)
library(ggplot2)
library(ggetho)
library(tidyverse)

# 2. load and link metadata to results folder

In [ ]:
metadata <- fread("/Users/Joana/sleep_signature/behaviour/10x/metadata.csv")
metadata$date <- as.character(metadata$date)
class(metadata$date)

metadata_10x <- link_ethoscope_metadata(metadata_10x,
                                              result_dir = "/ethoscope_data/results")
                                              
system.time(
  dt <- load_ethoscope(metadata,
                                 reference_hour=NA,
                                 FUN = sleepr::sleep_annotation,
                                 cache = "/ethoscope_data/cache",
                                 verbose=TRUE,
                                 map_arg=list(velocity_correction_coef="velocity_correction_coef")
  )
)

# 3. add phase column (light or dark phase) based on time

In [ ]:
dt[, phase := ifelse(t %% hours(24) < hours(12), "L", "D")]

# 4. population plot

In [ ]:
ggetho(dt, aes(y=asleep, colour=condition)) +
  ggetho::stat_pop_etho() +
  stat_ld_annotations(height = 1, alpha=0.1, outline=NA) +
  facet_grid(replicate ~ .) +
  theme_classic(base_size=8)+
  scale_y_continuous("Fraction of time asleep", limits=c(0,1), labels = scales::percent)+
  theme(axis.title.x=element_blank(),axis.text.x=element_blank(),axis.ticks.x=element_blank())+
  theme(axis.title.y=element_blank())+
  scale_colour_manual()+
  scale_fill_manual()

# 5. computing sleep duration during light and dark phases by fly ID

In [ ]:
# make new dt with calculations of mean during L and D

summary <-
  rejoin(dt[,
                .(
                  # sleep_fraction_all = mean(asleep),
                  sleep_fraction_l = mean(asleep[phase == "L"]),
                  sleep_fraction_d = mean(asleep[phase == "D"])
                ),
                ,by=id])
summary_dt

# make summary_dt_melted to make two variables: 'phase' and actual value in 'sleep_fraction'

summary_dt_melted <- melt(summary_dt, measure.vars = patterns("sleep_fraction_"),
                          variable.name = "phase", value.name = "sleep_fraction")


# 6. plotting sleep duration

In [ ]:
ggplot(summary_dt_melted, aes(x=replicate, y=sleep_fraction, colour=phase)) +
  geom_boxplot(outlier.colour = NA, varwidth = TRUE) +
  scale_y_continuous(name= "Fraction of time asleep",labels = scales::percent)+
  theme_bw(base_size = 15)

# 7. plotting motor rotations (interactions) as proxy of whether/how well sleep deprivation worked for each fly

In [ ]:
ggetho(dt, aes(y=interactions, colour=condition), summary_FUN = sum, summary_time_window = mins(30))+
  stat_pop_etho()+
  stat_ld_annotations(height = 1, alpha=0.1, outline=NA)+
  facet_grid(replicate ~ .)+
  theme_classic(base_size=10)+
  scale_colour_manual(values = "#16266B")+
  scale_fill_manual(values == "#16266B")